In [ ]:
from sklearn.preprocessing import MinMaxScaler
import IPython
import seaborn as sns
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import RepeatVector
from keras.layers import TimeDistributed, Conv1D
from keras.layers import Dropout
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from numpy import array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Preprocessing Data**

Don't run

In [ ]:
training_data_path = '/content/drive/MyDrive/orderbook_snapshots.csv'
df_training = pd.read_csv(training_data_path)
df_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7825199 entries, 0 to 7825198
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   2024-02-07 09:00:00  object 
 1   57                   int64  
 2   2367.3               float64
 3   2.03941              float64
 4   49_ask               object 
dtypes: float64(2), int64(1), object(2)
memory usage: 298.5+ MB


In [ ]:
output_filename = '/content/drive/My Drive/reformatted_depth3.csv'

orders = []
i = 0

# Read original csv and populate the orders dictionary
with open(training_data_path, mode='r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        timestamp, _, price, quantity, order_type = row
        if order_type == '0_ask':
            orders.append(row)
        if order_type == '0_bid':
            orders.append(row)
        if order_type == '1_ask':
            orders.append(row)
        if order_type == '1_bid':
            orders.append(row)
        if order_type == '2_ask':
            orders.append(row)
        if order_type == '2_bid':
            orders.append(row)
        i = i + 1

df = pd.DataFrame(orders, columns=['timestamp', 'id', 'price', 'qty', 'ask/bid'])
df

,timestamp,id,price,qty,ask/bid
0,2024-02-07 09:00:00,3,2361.9,7.27419703,2_ask
1,2024-02-07 09:00:00,2,2361.8,8.66562458,1_ask
2,2024-02-07 09:00:00,1,2361.7,5.481336570000001,0_ask
3,2024-02-07 09:00:00,534,2361.5,2.17785634,0_bid
4,2024-02-07 09:00:00,533,2361.4,8.548338039999999,1_bid
...,...,...,...,...,...
469507,2024-02-08 09:00:00,2,2423.6,15.00904452,1_ask
469508,2024-02-08 09:00:00,1,2423.5,0.27352833,0_ask
469509,2024-02-08 09:00:00,635,2423.4,1.55605682,0_bid
469510,2024-02-08 09:00:00,634,2423.3,4.48917427,1_bid


In [ ]:
split_df = np.array_split(df, 78252)
split_df

[             timestamp   id   price                qty ask/bid
 0  2024-02-07 09:00:00    3  2361.9         7.27419703   2_ask
 1  2024-02-07 09:00:00    2  2361.8         8.66562458   1_ask
 2  2024-02-07 09:00:00    1  2361.7  5.481336570000001   0_ask
 3  2024-02-07 09:00:00  534  2361.5         2.17785634   0_bid
 4  2024-02-07 09:00:00  533  2361.4  8.548338039999999   1_bid
 5  2024-02-07 09:00:00  532  2361.3        22.54287466   2_bid,
               timestamp   id   price                qty ask/bid
 6   2024-02-07 09:00:01    3  2361.9          6.6878074   2_ask
 7   2024-02-07 09:00:01    2  2361.8  7.265625920000001   1_ask
 8   2024-02-07 09:00:01    1  2361.7         7.73601583   0_ask
 9   2024-02-07 09:00:01  534  2361.5         2.31383667   0_bid
 10  2024-02-07 09:00:01  533  2361.4         8.28554801   1_bid
 11  2024-02-07 09:00:01  532  2361.3        22.08189947   2_bid,
               timestamp   id   price                 qty ask/bid
 12  2024-02-07 09:00:02    3

In [ ]:
'''
        timestamp         id   price            qty      ask/bid
 0  2024-02-07 09:00:00    3  2361.9         7.27419703   2_ask
 1  2024-02-07 09:00:00    2  2361.8         8.66562458   1_ask
 2  2024-02-07 09:00:00    1  2361.7  5.481336570000001   0_ask
 3  2024-02-07 09:00:00  534  2361.5         2.17785634   0_bid
 4  2024-02-07 09:00:00  533  2361.4  8.548338039999999   1_bid
 5  2024-02-07 09:00:00  532  2361.3        22.54287466   2_bid,
'''

new_arr = pd.DataFrame(columns=[ 'num', 'timestamp', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty'])

i = 0

for subset in split_df:
    row_0 = [0, subset['timestamp'].iloc[2], subset['price'].iloc[2], subset['qty'].iloc[2], subset['price'].iloc[3], subset['qty'].iloc[3]]
    row_1 = [1, subset['timestamp'].iloc[2], subset['price'].iloc[1], subset['qty'].iloc[1], subset['price'].iloc[4], subset['qty'].iloc[4]]
    row_2 = [2, subset['timestamp'].iloc[2], subset['price'].iloc[0], subset['qty'].iloc[0], subset['price'].iloc[5], subset['qty'].iloc[5]]
    new_arr.loc[len(new_arr.index)] = row_0
    new_arr.loc[len(new_arr.index)] = row_1
    new_arr.loc[len(new_arr.index)] = row_2
    i = i + 1
    print(i)

Streaming output truncated to the last 5000 lines.
73253
73254
73255
73256
73257
73258
73259
73260
73261
73262
73263
73264
73265
73266
73267
73268
73269
73270
73271
73272
73273
73274
73275
73276
73277
73278
73279
73280
73281
73282
73283
73284
73285
73286
73287
73288
73289
73290
73291
73292
73293
73294
73295
73296
73297
73298
73299
73300
73301
73302
73303
73304
73305
73306
73307
73308
73309
73310
73311
73312
73313
73314
73315
73316
73317
73318
73319
73320
73321
73322
73323
73324
73325
73326
73327
73328
73329
73330
73331
73332
73333
73334
73335
73336
73337
73338
73339
73340
73341
73342
73343
73344
73345
73346
73347
73348
73349
73350
73351
73352
73353
73354
73355
73356
73357
73358
73359
73360
73361
73362
73363
73364
73365
73366
73367
73368
73369
73370
73371
73372
73373
73374
73375
73376
73377
73378
73379
73380
73381
73382
73383
73384
73385
73386
73387
73388
73389
73390
73391
73392
73393
73394
73395
73396
73397
73398
73399
73400
73401
73402
73403
73404
73405
73406
73407
73408
73409
73410
7

In [ ]:
new_arr

,num,timestamp,best_ask_price,best_ask_qty,best_bid_price,best_bid_qty
0,0,2024-02-07 09:00:00,2361.7,5.481336570000001,2361.5,2.17785634
1,1,2024-02-07 09:00:00,2361.8,8.66562458,2361.4,8.548338039999999
2,2,2024-02-07 09:00:00,2361.9,7.27419703,2361.3,22.54287466
3,0,2024-02-07 09:00:01,2361.7,7.73601583,2361.5,2.31383667
4,1,2024-02-07 09:00:01,2361.8,7.265625920000001,2361.4,8.28554801
...,...,...,...,...,...,...
234751,1,2024-02-08 08:59:59,2423.6,15.00904452,2423.3,3.7496218900000002
234752,2,2024-02-08 08:59:59,2423.7,4.69398015,2423.2,7.20200426
234753,0,2024-02-08 09:00:00,2423.5,0.27352833,2423.4,1.55605682
234754,1,2024-02-08 09:00:00,2423.6,15.00904452,2423.3,4.48917427


In [ ]:
new_arr.to_csv('/content/drive/My Drive/depth3_orderbooks.csv')

## **Preprocessing test data**

Don't run

In [ ]:
test_data_path = '/content/drive/MyDrive/ds.csv'
df_test = pd.read_csv(test_data_path)
df_test

output_filename = '/content/drive/My Drive/depth3_testdata.csv'

orders = []
i = 0

# Read original csv and populate the orders dictionary
with open(test_data_path, mode='r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        timestamp, _, price, quantity, order_type = row
        if order_type == '0_ask':
            orders.append(row)
        if order_type == '0_bid':
            orders.append(row)
        if order_type == '1_ask':
            orders.append(row)
        if order_type == '1_bid':
            orders.append(row)
        if order_type == '2_ask':
            orders.append(row)
        if order_type == '2_bid':
            orders.append(row)
        i = i + 1

df_test = pd.DataFrame(orders, columns=['timestamp', 'id', 'price', 'qty', 'ask/bid'])
df_test

,timestamp,id,price,qty,ask/bid
0,2024-02-12 14:29:32,3,2495.1,6.87228636,2_ask
1,2024-02-12 14:29:32,2,2495.0,4.91982168,1_ask
2,2024-02-12 14:29:32,1,2494.9,1.62143945,0_ask
3,2024-02-12 14:29:32,474,2494.8,3.86355557,0_bid
4,2024-02-12 14:29:32,473,2494.7,1.10487831,1_bid
...,...,...,...,...,...
308317,2024-02-13 14:51:24,3,2630.5,5.62238611,1_ask
308318,2024-02-13 14:51:24,1,2630.3,1.8690473699999999,0_ask
308319,2024-02-13 14:51:24,873,2630.0,2.14899681,0_bid
308320,2024-02-13 14:51:24,872,2629.9,0.04662098,1_bid


In [ ]:
split_df_test = np.array_split(df_test, 51387)
split_df_test

[             timestamp   id   price          qty ask/bid
 0  2024-02-12 14:29:32    3  2495.1   6.87228636   2_ask
 1  2024-02-12 14:29:32    2  2495.0   4.91982168   1_ask
 2  2024-02-12 14:29:32    1  2494.9   1.62143945   0_ask
 3  2024-02-12 14:29:32  474  2494.8   3.86355557   0_bid
 4  2024-02-12 14:29:32  473  2494.7   1.10487831   1_bid
 5  2024-02-12 14:29:32  472  2494.6  26.85138914   2_bid,
               timestamp   id   price          qty ask/bid
 6   2024-02-12 14:29:33    3  2495.1   3.86636934   2_ask
 7   2024-02-12 14:29:33    2  2495.0   4.91982168   1_ask
 8   2024-02-12 14:29:33    1  2494.9   1.42594843   0_ask
 9   2024-02-12 14:29:33  474  2494.8   2.26023265   0_bid
 10  2024-02-12 14:29:33  473  2494.7   3.79671688   1_bid
 11  2024-02-12 14:29:33  472  2494.6  26.56279178   2_bid,
               timestamp   id   price                qty ask/bid
 12  2024-02-12 14:29:34    3  2495.1  4.609110380000001   2_ask
 13  2024-02-12 14:29:34    2  2495.0         8.7

In [ ]:
'''
        timestamp         id   price            qty      ask/bid
 0  2024-02-07 09:00:00    3  2361.9         7.27419703   2_ask
 1  2024-02-07 09:00:00    2  2361.8         8.66562458   1_ask
 2  2024-02-07 09:00:00    1  2361.7  5.481336570000001   0_ask
 3  2024-02-07 09:00:00  534  2361.5         2.17785634   0_bid
 4  2024-02-07 09:00:00  533  2361.4  8.548338039999999   1_bid
 5  2024-02-07 09:00:00  532  2361.3        22.54287466   2_bid,
'''

new_arr_test = pd.DataFrame(columns=[ 'num', 'timestamp', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty'])

i = 0

for subset in split_df_test:
    row_0 = [0, subset['timestamp'].iloc[2], subset['price'].iloc[2], subset['qty'].iloc[2], subset['price'].iloc[3], subset['qty'].iloc[3]]
    row_1 = [1, subset['timestamp'].iloc[2], subset['price'].iloc[1], subset['qty'].iloc[1], subset['price'].iloc[4], subset['qty'].iloc[4]]
    row_2 = [2, subset['timestamp'].iloc[2], subset['price'].iloc[0], subset['qty'].iloc[0], subset['price'].iloc[5], subset['qty'].iloc[5]]
    new_arr_test.loc[len(new_arr_test.index)] = row_0
    new_arr_test.loc[len(new_arr_test.index)] = row_1
    new_arr_test.loc[len(new_arr_test.index)] = row_2
    i = i + 1
    print(i)

Streaming output truncated to the last 5000 lines.
46388
46389
46390
46391
46392
46393
46394
46395
46396
46397
46398
46399
46400
46401
46402
46403
46404
46405
46406
46407
46408
46409
46410
46411
46412
46413
46414
46415
46416
46417
46418
46419
46420
46421
46422
46423
46424
46425
46426
46427
46428
46429
46430
46431
46432
46433
46434
46435
46436
46437
46438
46439
46440
46441
46442
46443
46444
46445
46446
46447
46448
46449
46450
46451
46452
46453
46454
46455
46456
46457
46458
46459
46460
46461
46462
46463
46464
46465
46466
46467
46468
46469
46470
46471
46472
46473
46474
46475
46476
46477
46478
46479
46480
46481
46482
46483
46484
46485
46486
46487
46488
46489
46490
46491
46492
46493
46494
46495
46496
46497
46498
46499
46500
46501
46502
46503
46504
46505
46506
46507
46508
46509
46510
46511
46512
46513
46514
46515
46516
46517
46518
46519
46520
46521
46522
46523
46524
46525
46526
46527
46528
46529
46530
46531
46532
46533
46534
46535
46536
46537
46538
46539
46540
46541
46542
46543
46544
46545
4

In [ ]:
new_arr_test

,num,timestamp,best_ask_price,best_ask_qty,best_bid_price,best_bid_qty
0,0,2024-02-12 14:29:32,2494.9,1.62143945,2494.8,3.86355557
1,1,2024-02-12 14:29:32,2495.0,4.91982168,2494.7,1.10487831
2,2,2024-02-12 14:29:32,2495.1,6.87228636,2494.6,26.85138914
3,0,2024-02-12 14:29:33,2494.9,1.42594843,2494.8,2.26023265
4,1,2024-02-12 14:29:33,2495.0,4.91982168,2494.7,3.79671688
...,...,...,...,...,...,...
154156,1,2024-02-13 14:51:22,2629.8,0.65811385,2629.3,0.68230737
154157,2,2024-02-13 14:51:22,2629.9,1.09685626,2629.2,1.36486678
154158,0,2024-02-13 14:51:24,2630.3,1.8690473699999999,2630.0,2.14899681
154159,1,2024-02-13 14:51:24,2630.5,5.62238611,2629.9,0.04662098


In [ ]:
new_arr_test.to_csv('/content/drive/My Drive/depth3_orderbooks_fortesting.csv')

## **Load data from csv**

In [ ]:
data_path = '/content/drive/My Drive/depth3_orderbooks.csv'
columns = ['num', 'timestamp', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']
data = pd.read_csv(data_path, header=None, names=columns, skiprows=1)

data

,num,timestamp,best_ask_price,best_ask_qty,best_bid_price,best_bid_qty
0,0,2024-02-07 09:00:00,2361.7,5.481337,2361.5,2.177856
1,1,2024-02-07 09:00:00,2361.8,8.665625,2361.4,8.548338
2,2,2024-02-07 09:00:00,2361.9,7.274197,2361.3,22.542875
3,0,2024-02-07 09:00:01,2361.7,7.736016,2361.5,2.313837
4,1,2024-02-07 09:00:01,2361.8,7.265626,2361.4,8.285548
...,...,...,...,...,...,...
234751,1,2024-02-08 08:59:59,2423.6,15.009045,2423.3,3.749622
234752,2,2024-02-08 08:59:59,2423.7,4.693980,2423.2,7.202004
234753,0,2024-02-08 09:00:00,2423.5,0.273528,2423.4,1.556057
234754,1,2024-02-08 09:00:00,2423.6,15.009045,2423.3,4.489174


In [ ]:
# Convert timestamp to a numerical format
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y.%m.%d %H:%M:%S')

# Normalize the dataset
scaler = MinMaxScaler()
data[['num', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']] = \
    scaler.fit_transform(data[['num', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']])

data.head()

,num,timestamp,best_ask_price,best_ask_qty,best_bid_price,best_bid_qty
0,0.0,2024-02-07 09:00:00,0.097222,0.006911,0.102674,0.005423
1,0.5,2024-02-07 09:00:00,0.098291,0.010926,0.101604,0.021286
2,1.0,2024-02-07 09:00:00,0.099359,0.009171,0.100535,0.056134
3,0.0,2024-02-07 09:00:01,0.097222,0.009753,0.102674,0.005762
4,0.5,2024-02-07 09:00:01,0.098291,0.009160,0.101604,0.020632


In [ ]:
x = data[0:230000]
in_seq00 = x[['timestamp']]
in_seq0 = x[['num']]
in_seq1 = x[['best_ask_price']]
in_seq2 = x[['best_ask_qty']]
in_seq3 = x[['best_bid_price']]
in_seq4 = x[['best_bid_qty']]

x

,num,timestamp,best_ask_price,best_ask_qty,best_bid_price,best_bid_qty
0,0.0,2024-02-07 09:00:00,0.097222,0.006911,0.102674,0.005423
1,0.5,2024-02-07 09:00:00,0.098291,0.010926,0.101604,0.021286
2,1.0,2024-02-07 09:00:00,0.099359,0.009171,0.100535,0.056134
3,0.0,2024-02-07 09:00:01,0.097222,0.009753,0.102674,0.005762
4,0.5,2024-02-07 09:00:01,0.098291,0.009160,0.101604,0.020632
...,...,...,...,...,...,...
229995,0.0,2024-02-08 08:15:21,0.766026,0.011042,0.773262,0.015826
229996,0.5,2024-02-08 08:15:21,0.767094,0.002080,0.772193,0.012381
229997,1.0,2024-02-08 08:15:21,0.769231,0.024633,0.771123,0.018161
229998,0.0,2024-02-08 08:15:22,0.766026,0.012892,0.773262,0.016134


In [ ]:
from keras.layers import (Conv1D, MaxPool1D, Dropout, Flatten, Dense,
                          Input, concatenate)
from keras.models import Model, Sequential

timesteps = 3
n_features = 5
n_input = 3

def network():
    sequence = Input(shape=(timesteps, n_features), name='Sequence')
    features = Input(shape=(n_features,), name='Features')

    conv = Sequential()
    conv.add(LSTM(64, input_shape=(n_input, n_features)))
    conv.add((Dense(6, activation='leaky_relu')))

    part1 = conv(sequence)

    merged = concatenate([part1, features])

    final = Dense(512, activation='relu')(merged)
    final = Dropout(0.5, seed=789)(final)
    final = Dense(5, activation='softmax')(final)

    model = Model(inputs=[sequence, features], outputs=[final])

    model.compile(loss='logcosh', optimizer='adam', metrics=['accuracy'])

    return model

m = network()

In [ ]:
# horizontally stack columns
dataset = hstack((in_seq0, in_seq1, in_seq2, in_seq3, in_seq4))
print(dataset)

[[0.         0.09722222 0.0069108  0.1026738  0.00542313]
 [0.5        0.0982906  0.01092552 0.10160428 0.0212864 ]
 [1.         0.09935897 0.00917122 0.10053476 0.05613449]
 ...
 [1.         0.76923077 0.02463285 0.77112299 0.0181611 ]
 [0.         0.76602564 0.01289218 0.77326203 0.01613378]
 [0.5        0.76709402 0.00348439 0.77219251 0.0033295 ]]


In [ ]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Sequence (InputLayer)       [(None, 3, 5)]               0         []                            
                                                                                                  
 sequential (Sequential)     (None, 6)                    18310     ['Sequence[0][0]']            
                                                                                                  
 Features (InputLayer)       [(None, 5)]                  0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 11)                   0         ['sequential[0][0]',          
                                                                     'Features[0][0]']        

In [ ]:
n_features = 6
n_input = 3
data_list = tuple(dataset)

generator = TimeseriesGenerator(data_list, data_list, length=n_input, batch_size=4)

m.compile(loss='mse', optimizer='adam', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

m.fit(data_list, steps_per_epoch=3, epochs=1, verbose=0)

NameError: name 'dataset' is not defined

In [ ]:
# Print some samples
print('Samples: %d' % len(generator))
for i in range(0,20):
 x, y = generator[i]
 print('%s => %s' % (x, y))

In [ ]:
# next three rows should be : [0, 0.097222, 0.006911, 0.102674, 0.005423], [0.098291, 0.010926,	0.101604, 0.021286], [0.099359, 0.009171, 0.100535, 0.056134]
x_input = array([ [0, 0.097222,	0.006911,	0.102674,	0.005423], [0.5, 0.098291, 0.010926,	0.101604,	0.021286], [1.0, 0.099359,	0.009171,	0.100535,	0.056134]  ]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
yhat

In [ ]:
arr = []
i = 0

def timeseries(a, b, c, i):
  arr.append([a,b,c])
  x_input = array([a, b, c]).reshape((1, n_input, n_features))
  yhat = model.predict(x_input, verbose=0)
  yhat.tolist
  x_input_2 = array([b, c, yhat[0] ]).reshape((1, n_input, n_features))
  yhat_2 = model.predict(x_input_2, verbose=0)
  yhat_2.tolist
  x_input_3 = array([c, yhat[0], yhat_2[0] ]).reshape((1, n_input, n_features))
  yhat_3 = model.predict(x_input_3, verbose=0)
  yhat_3.tolist
  return timeseries(yhat[0], yhat_2[0], yhat_3[0], i)

for i in range(0, 60):
  a, b, c = timeseries([0, 0.097222, 0.006911, 0.102674, 0.005423], [0.5, 0.098291, 0.010926,	0.101604, 0.021286], [1.0, 0.099359, 0.009171, 0.100535, 0.056134], i)

print((a, b, c))

In [ ]:
import math
import itertools
from itertools import cycle

log = []

for item in arr:
  a = item[0]
  b = item[1]
  c = item[2]
  A = array(a)
  yhat_0 = \
    scaler.inverse_transform(A.reshape(1, -1))
  yhat_result_0 = pd.DataFrame(yhat_0, index=None)
  B = array(b)
  yhat_1 = \
    scaler.inverse_transform(B.reshape(1, -1))
  yhat_result_1 = pd.DataFrame(yhat_1, index=None)
  C = array(c)
  yhat_2 = \
    scaler.inverse_transform(C.reshape(1, -1))
  yhat_result_2 = pd.DataFrame(yhat_2, index=None)
  temp = [0, yhat_result_0[1][0], yhat_result_0[2][0], yhat_result_0[3][0],  yhat_result_0[4][0]]
  log.append(temp)
  temp = [1, yhat_result_1[1][0], yhat_result_1[2][0], yhat_result_1[3][0],  yhat_result_1[4][0]]
  log.append(temp)
  temp = [2, yhat_result_2[1][0], yhat_result_2[2][0], yhat_result_2[3][0], yhat_result_2[4][0]]
  log.append(temp)

log_results = pd.DataFrame(log, columns=['bid/ask Number', 'ask_price', 'ask_qty', 'bid_price', 'bid_qty'])
log_results

In [ ]:
# @title ask_qty vs bid_price

from matplotlib import pyplot as plt
log_results.plot(kind='scatter', x='ask_qty', y='bid_price', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ask_price']
  ys = series['ask_qty']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = log_results.sort_values('ask_price', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ask_price')
_ = plt.ylabel('ask_qty')

In [ ]:
(log_results.values < 0).any()


In [ ]:
# TESTING Index 62603 and 62604 -> 62605
# Values next in sequence should be : 0.007228	0.004854	0.007752	0.001148
x_input = array([ [0.006744, 0.002781, 0.007236, 0.009131], [0.007744, 0.006781, 0.007235, 0.008121], [0, 0.006711, 0.001407, 0.008269, 0.000684]]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
yhat

In [ ]:
test_data_path = '/content/drive/My Drive/depth3_orderbooks_fortesting.csv'
columns = ['num', 'timestamp', 'best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']
test_data = pd.read_csv(test_data_path, header=None, names=columns, skiprows=1)

# Convert timestamp to a numerical format
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'], format='%Y.%m.%d %H:%M:%S')

date_time = test_data.pop('timestamp')

# Normalize the dataset
scaler = MinMaxScaler()
test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']] = \
    scaler.fit_transform(test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']])

slice_length = 5000
particular_index = test_data.iloc[8001].name
test_data = test_data.loc[:particular_index].tail(slice_length)

test_data

In [ ]:
test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']] = \
    scaler.inverse_transform(test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']])

test_df_reversed = pd.DataFrame(test_data, index=None)
test_df_reversed

In [ ]:
test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']] = \
    scaler.fit_transform(test_data[['best_ask_price', 'best_ask_qty', 'best_bid_price', 'best_bid_qty']])

test_data

In [ ]:
# Use two values from the test dataset, to predict the next, then compare
arr = []
i = 0
for (indx1,row1),(indx2,row2),(indx3,row3), (indx4,row4) in zip(
    test_data[:-3].iterrows(),test_data[1:-1].iterrows(),test_data[2:-2].iterrows(),test_data[3:].iterrows()):
    temp1 = [row1['best_ask_price'], row1['best_ask_qty'], row1['best_bid_price'], row1['best_bid_qty']]
    temp2 = [row2['best_ask_price'], row2['best_ask_qty'], row2['best_bid_price'], row2['best_bid_qty']]
    temp3 = [row3['best_ask_price'], row3['best_ask_qty'], row3['best_bid_price'], row3['best_bid_qty']]
    temp4 = [row4['best_ask_price'], row4['best_ask_qty'], row4['best_bid_price'], row4['best_bid_qty']]
    x_input = array([temp1, temp2, temp3]).reshape((1, n_input, n_features))
    yhat = model.predict(x_input, verbose=0)
    arr.append([temp4, yhat])
    print(i)
    i = i + 1

In [ ]:
mean_squared_errors = []

i = 0

for item in arr:
  real = item[0]
  prediction = item[1][0]
  mse = mean_squared_error(real, prediction)
  mean_squared_errors.append(mse)
  i = i + 1

average_mse = np.mean(mean_squared_errors)
print("Average mean squared error: ", f'{average_mse:.20f}')

In [ ]:
import math

# Create a df showing the predicted vs real values
# Ask and bid prices are quite accurate, Ask and bid quantities are quite off
i = 0

log = []

for item in arr:
  real = item[0]
  prediction = item[1][0]
  B = array([prediction])
  yhat = \
    scaler.inverse_transform(B)
  yhat_result = pd.DataFrame(yhat, index=None)
  A = array([real])
  yhat_real = \
    scaler.inverse_transform(A)
  yhat_real = pd.DataFrame(yhat_real, index=None)
  temp = [ yhat_real[0][0], yhat_result[0][0], yhat_real[1][0], yhat_result[1][0], yhat_real[2][0], yhat_result[2][0], yhat_real[3][0], yhat_result[3][0] ]
  log.append(temp)

log_results = pd.DataFrame(log, columns=['real_ask_price', 'pred_ask_price', 'real_ask_qty', 'pred_ask_qty', 'real_bid_price', 'pred_bid_price', 'real_bid_qty', 'pred_bid_qty'])
log_results

In [ ]:
(log_results.values < 0).any()

import itertools
from itertools import cycle

num_cycle = cycle([0, 1, 2])
log_results['Bid/Ask Number'] = [next(num_cycle) for num in range(len(log_results))]


In [ ]:
(log_results.values < 0).any()


In [ ]:
log_results

In [ ]:
from matplotlib import pyplot as plt
log_results['real_ask_qty'].plot(kind='line', figsize=(8, 4), title='pred vs. real ask_qty', label="real")

log_results['pred_ask_qty'].plot(kind='line', figsize=(8, 4), label="predicted")

plt.legend()

plt.gca().spines[['top', 'right']].set_visible(False)


In [ ]:
from matplotlib import pyplot as plt
log_results['real_bid_qty'].plot(kind='line', figsize=(8, 4), title='pred vs. real bid_qty', label="real")

log_results['pred_bid_qty'].plot(kind='line', figsize=(8, 4), label="predicted")

plt.legend()

plt.gca().spines[['top', 'right']].set_visible(False)


In [ ]:
# Real Ask Price vs. Predicted Ask Price

from matplotlib import pyplot as plt
log_results['real_ask_price'].plot(kind='line', figsize=(8, 4), title='Real vs. Pred - Ask Price')
plt.plot(log_results['pred_ask_price'], alpha=0.6, label="predicted_ask_price")

plt.legend()

plt.gca().spines[['top', 'right']].set_visible(False)


In [ ]:
# Real Ask Quantity vs Predicted Ask Quantity
from matplotlib import pyplot as plt

log_results.plot(kind='scatter', x='real_ask_qty', y='pred_ask_qty', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Real Ask Quantity vs Predicted Ask Quantity
from matplotlib import pyplot as plt

log_results.plot(kind='scatter', x='real_bid_qty', y='pred_bid_qty', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)